In [ ]:
import tkinter as tk
# TODO: Combine all three GUI into one GUI (File import, County BUS, Checkbox)
# Create the main window
root = tk.Tk()
root.title("Checkbox Example")

# Variable to store the checkbox state
checkbox_AUX = tk.BooleanVar()
checkbox_N_2 = tk.BooleanVar()
checkbox_N_1_1 = tk.BooleanVar()
checkbox_PO = tk.BooleanVar()

title = f"Check the files you want to save during the process. \n If none of the options are checked, \n only maximum violation csv file will be saved"

label = tk.Label(root, text=title)
label.pack(anchor=tk.W)
# Create a checkbox

checkbox1 = tk.Checkbutton(root, text="Save AUX File", variable=checkbox_AUX)
checkbox1.pack(anchor="w")

checkbox2 = tk.Checkbutton(root, text="Save N-2 case and result File", variable=checkbox_N_2)
checkbox2.pack(anchor="w")

checkbox3 = tk.Checkbutton(root, text="Save N-1-1 case and result File", variable=checkbox_N_1_1)
checkbox3.pack(anchor="w")

checkbox4 = tk.Checkbutton(root, text="Save PO cases", variable=checkbox_PO)
checkbox4.pack(anchor="w")

# Function to display the checkbox state
def show_selection():
    selected = []

    selected.append(f"Option AUX = {checkbox_AUX.get()}")

    selected.append(f"Option N_2 = {checkbox_N_2.get()}")

    selected.append(f"Option N_1_1 = {checkbox_N_1_1.get()}")

    selected.append(f"Option PO = {checkbox_PO.get()}")
    print("Selected:", ", ".join(selected))
        
# Function to close the window
def close_window():
    root.destroy()


# Button to check the state
submit_button = tk.Button(root, text="Submit", command=close_window)
submit_button.pack()


# Run the application
root.mainloop()

checkbox_AUX.get(), checkbox_N_2.get(), checkbox_N_1_1.get(), checkbox_PO.get()

Selected file: O:/Project/Project vault/Baytown/3wl4_Opt2A_Maint.pwb


In [ ]:
import tkinter as tk
from tkinter import ttk
# ! Problem, everytime I type, it interrupt typing for some reason. Fix this issue
class AutocompleteCombobox(ttk.Combobox):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self._completion_list = []
        self.bind('<KeyRelease>', self._on_keyrelease)

    def set_completion_list(self, completion_list):
        self._completion_list = sorted(completion_list)
        self['values'] = self._completion_list

    def _on_keyrelease(self, event):
        value = self.get()
        if value == '':
            self['values'] = self._completion_list
        else:
            filtered = [item for item in self._completion_list if value.lower() in item.lower()]
            self['values'] = filtered
        self.event_generate('<Down>')  # Open dropdown

# Main application
def main():
    root = tk.Tk()
    root.title("Two Autocomplete Comboboxes")

    # First Combobox
    label1 = ttk.Label(root, text="Select County:")
    label1.pack(pady=5)
    combobox1 = AutocompleteCombobox(root)
    combobox1.pack(pady=5)
    combobox1.set_completion_list(county['COUNTY'].unique())

    # Second Combobox
    label2 = ttk.Label(root, text="Select BusNumber:")
    label2.pack(pady=5)
    combobox2 = AutocompleteCombobox(root)
    combobox2.pack(pady=5)
    combobox2.set_completion_list(list(bus['BusNum'].astype(str)))

    root.mainloop()

if __name__ == "__main__":
    main()


NameError: name 'file_path' is not defined

In [ ]:
import tkinter as tk
from tkinter import ttk
from tkinterdnd2 import DND_FILES, TkinterDnD
import os
import string

# $ RTP Team's request:
# $ 1. Add another GUI for County and additional Bus number input.
# $ 2. Instead of display Option A B C, have a string box for naming. Default = Option A B C
# $ 3. Check box GUI user interface for output save from each function. Default = everything unchecked. (Only save the result of the M.O)
class FileDropApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Drag and Drop File Uploader")

        self.frame = ttk.Frame(root, padding=10)
        self.frame.pack(fill=tk.BOTH, expand=True)

        self.option_counter = 0
        self.listboxes = []
        self.result = None  # Store result to return later

        self.add_option_box()

        # Add buttons
        btn_frame = ttk.Frame(self.frame)
        btn_frame.pack(pady=10)

        add_btn = ttk.Button(btn_frame, text="Add Option Box", command=self.add_option_box)
        add_btn.pack(side=tk.LEFT, padx=5)

        finish_btn = ttk.Button(btn_frame, text="Finish Upload", command=self.handle_finish_upload)
        finish_btn.pack(side=tk.LEFT, padx=5)

    def add_option_box(self):

        # replace with string input box
        title = f"Option {string.ascii_uppercase[self.option_counter]}"
        self.option_counter += 1

        label = ttk.Label(self.frame, text=title)
        label.pack(anchor=tk.W)

        listbox = tk.Listbox(self.frame, selectmode=tk.MULTIPLE, width=80, height=5)
        listbox.pack(pady=5, fill=tk.X)

        listbox.drop_target_register(DND_FILES)
        listbox.dnd_bind('<<Drop>>', lambda e, lb=listbox: self.on_drop(e, lb))

        self.listboxes.append((title, listbox))

    def on_drop(self, event, listbox):
        files = self.root.tk.splitlist(event.data)
        for file in files:
            if os.path.isfile(file) and file not in listbox.get(0, tk.END):
                listbox.insert(tk.END, file)

    def handle_finish_upload(self):
        result = []
        for title, lb in self.listboxes:
            file_list = [lb.get(i) for i in range(lb.size())]
            result.append((title, file_list))

        self.result = result
        self.root.quit()  # Close the Tkinter loop but keep the app object

def run_app():
    root = TkinterDnD.Tk()
    app = FileDropApp(root)
    root.mainloop()
    root.destroy()  # Fully close window
    return app.result  # Return the result

# Example of calling the app and using the result
# ! Check the content inside the box, if user upload other files than PWB or AUX file or upload more than one PWB or AUX file. Raise an error and
# ! reset the box
if __name__ == "__main__":
    uploaded_files = run_app()
    print("\nReturned Data Structure:")
    for option, files in uploaded_files:
        print(f"{option}:")
        for f in files:
            print(f"  {f}")
        if not files:
            print("  [No files uploaded]")
    
    print(uploaded_files[0][0],uploaded_files[0][1])


<tkinter.ttk.Button object .!button>

In [23]:
import os

file = r"O:\Project\Project vault\Baytown\3wl4_Opt2A_Maint.pwb"
direcotry = os.path.dirname(file) + "\\"
direcotry

'O:\\Project\\Project vault\\Baytown\\'